# Use Case Machine Learning

>



---



# Installation des packages nécessaires pour les plots.

In [ ]:
! pip install --upgrade plotly
! pip install jupyter-dash

In [ ]:
! pip install kneed

# Importation des classes nécessaires pour notre étude.

In [ ]:
import pandas as pd
import numpy as np

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff

from sklearn.impute import KNNImputer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from kneed import KneeLocator # sert à détecter l'hyper-parametre K pour la méthode KMeans
from scipy.cluster.hierarchy import dendrogram, linkage
from sklearn.cluster import AgglomerativeClustering

from sklearn.metrics import silhouette_score

# Chargement des données

In [ ]:
! gdown --id 1f3saJ6cefP_UUSZ4Qi9M488rNrU6nSS1

In [ ]:
df = pd.read_csv('base-comparateur-de-territoires.csv', sep = ";")

# Exploration

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

On a 3 observations qui n'ont pas des coordonnées, on ne peut pas les donner des valeurs aléatoires.

In [ ]:
df.info()

On doit les supprimer.

In [ ]:
df.dropna(subset = ['geo_point_2d'], inplace = True)

On détache le libelé et les coordonnées pour les utiliser après.

In [ ]:
libele = df['LIBGEO']
lat = [float(x.split(",")[0]) for x in df['geo_point_2d'] if not pd.isnull(x)]
lon = [float(x.split(",")[1]) for x in df['geo_point_2d'] if not pd.isnull(x)]

Suppréssion des features inutiles (on est interessé par les données sociodémographiques)

In [ ]:
df.drop(['CODGEO', 'LIBGEO', 'DEP', 'REG', 'Geo Shape', 'geo_point_2d'], axis = 1, inplace = True)

Il reste encore des valeurs manquants.

In [ ]:
df.isna().sum()

Suppréssion des features qui ont un pourcentage des valeurs manquantes supérieur à 30%.
* Pandas.count() -> compte nombre de valeurs non nulles d'une colonne.

In [ ]:
nrows = df.shape[0]
for col in df.columns:
  if ( nrows - df[col].count() ) / nrows > 0.3:
    print(f'La colonne {col} a été supprimé.')
    df.drop(col, axis = 1, inplace = True)

Remplissage des valeurs manquantes par la méthode KNNImputer.

In [ ]:
for col in df.columns:
  imputer = KNNImputer(missing_values = np.nan, n_neighbors = 3)
  df[col] = imputer.fit_transform(df[col].values.reshape(-1, 1))

Box plot pour vérifier l'existence des valeurs outliers.
- On remarque qu'il existe des outliers dans toutes les colonnes.

In [ ]:
px.box(df)

Utilisation de la méthode 'Interquartile range' (IQR) pour l'élimination des outliers.
* numpy.where(condition, x, y) 
  * si le condition est vrai retourne x sinon y.

In [ ]:
for col in df.columns:
  q25, q75 = np.percentile(df[col].dropna(), 25), np.percentile(df[col].dropna(), 75)
  iqr = q75 - q25
  upper_limit = q75 + 1.5 * iqr
  lower_limit = q25 - 1.5 * iqr
  df[col] = np.where(
      df[col] < lower_limit,
      lower_limit,
      np.where(
          df[col] > upper_limit,
          upper_limit,
          df[col]
      )
  )

Les outliers ont été supprimés avec succès.
- Les features ont des unités différent, on doit les standariser.

In [ ]:
px.box(df)

# Test de corrélation

Heat map des corrélations entre les features.
- On remarque que la plupart des features sont fortement corrélés entre eux.
 - On doit réduire la dimensionnalité.

In [ ]:
fig = go.Figure(
    data = go.Heatmap(
        z = df.corr(), x = df.columns, y = df.columns
    )
)
fig.show()

# Standarisation

Méthode de z-score.
- (Xi - moyen) / écart-type

In [ ]:
df_scaled = StandardScaler().fit_transform(df)

# ACP

Réduction de dimensionnalité.

In [ ]:
df_pca = PCA().fit(df_scaled)

Calcul de la somme cummulé des variances exprimés par les features afin de détecter le nombre de composants qui expriment le plus part d'information.

In [ ]:
cumsum_variance = np.cumsum(df_pca.explained_variance_ratio_)

Plot de sommes cummulés des variances.
- kteb lih comenter a amhil rak tema (3zelna nb components = 2 hitach qui exprimer ~95% dyl l'information)

In [ ]:
px.line(cumsum_variance)

ACP avec 2 composants principales.

In [ ]:
df_pca = PCA(n_components=2).fit_transform(df)

In [ ]:
df_pca



> # KMeans



### Détection de la meilleur valeur d'hyper-parametre K

inertie = somme carré des distances entre les observations
- On sauvegarde les models correpondants aux différents valeurs de K pour ne pas les calculer à nouveau lors après le choix de K optimale.

In [ ]:
nb_clusters = range(2, 31)
inertie = []
models = []
for K in nb_clusters:
  kmeans = KMeans(K)
  kmeans.fit(df_pca)
  models.append(kmeans)
  inertie.append(kmeans.inertia_)

Elbow plot.
- La meilleure valeur de K est comprise entre 7 et 10.

In [ ]:
px.line(
    x = nb_clusters,
    y = inertie,
    labels = {
        "x": "nombre de clusters (K)",
        "y": "inertie"
    }
)

Détection numérique de 'Elbow point' avec la fonction KneeLocator du package 'kneed'.

In [ ]:
kl = KneeLocator(nb_clusters, inertie, S=1.0, curve='convex', direction='decreasing')
K = kl.elbow
print(f'La valeur optimale de K est {K}.')

**range(2, ..)** donc le model correspondant est K - 2.

In [ ]:
kmeans = models[K - 2]

In [ ]:
kmeans.cluster_centers_

In [ ]:
classes_kmeans = kmeans.labels_
classes_kmeans


> # CAH



Dendrogram
- Le K est le nombre des lignes verticales qui traversent la coupe horizontale correspondant au plus grand saut entre deux clusters consécutifs.
- Ward's Linkage Method. Instead of measuring the distance directly, it analyzes the variance of clusters. Ward's is said to be the most suitable method for quantitative variables.

In [ ]:
ff.create_dendrogram(
    df_pca,
    linkagefun = lambda x: linkage(x, "ward")
)

In [ ]:
cah = AgglomerativeClustering(n_clusters = 5).fit(df_pca)

In [ ]:
classes_cah = cah.labels_
classes_cah


> # Comparaison entre KMeans et CAH






- On ne peut pas évaluer nos models puisqu'on n'a pas des classes (labels) réels pour notre données.
 - Donc Il faut comparer le coefficient silhouette des deux clusters générés par les deux models.
   - The silhouette method computes silhouette coefficients of each point that measure how much a point is similar to its own cluster compared to other clusters.

Le coefficient silhouette de CAH.

In [ ]:
silhouette_score(df_pca, classes_cah)

Le coefficient silhouette de KMeans.

In [ ]:
silhouette_score(df_pca, kmeans.labels_)

Le coefficient de la méthode KMeans est plus grand que celui de CAH.
- On va utiliser le KMeans pour le regroupement des communes.

Assemblage de données.

In [ ]:
communes = pd.DataFrame(
    {
      'component1': df_pca[:, 0],
      'component2': df_pca[:, 1],
      'classe': kmeans.labels_,
      'libele': libele,
      'lat': lat,
      'lon': lon
    }
)

In [ ]:
communes.head()

10 communes qui ont des caractéristiques communs et qui appartient au 1er groupement.

In [ ]:
communes[communes['classe'] == 0]['libele'].tolist()[:10]


> # Visualisation



Visualisation des communes avec le plot à nuage de points.

In [ ]:
px.scatter(
    communes,
    x = 'component1',
    y = 'component2',
    color = 'classe',
    hover_data = ['libele']
)

Visualisation des communes dans un Map.
- color = appartenance à une classe.

In [ ]:
px.scatter_mapbox(
    communes,
    lat = "lat",
    lon = "lon",
    color = "classe",
    hover_data = ['libele'],
    size_max = 20,
    zoom = 9,
    mapbox_style = "carto-positron"
)

Map plus sophistiqué.

In [ ]:
px.scatter_mapbox(
    communes,
    lat = "lat",
    lon = "lon",
    color = "classe",
    hover_data = ['libele'],
    size_max = 20,
    zoom = 9,
    mapbox_style = "open-street-map"
)